# --------- NDVI and DNBR for 2020 fires in Angola --------- #

# 0. Data acquisition 

Prior to running this code, the following datasets should be downloaded: 

1) Globfire (Artés et al., 2019), available here: https://gwis.jrc.ec.europa.eu/apps/country.profile/downloads

   *Note: the dataset used here is a subset of the Globfire dataset, created by Brittany Engle, containing only 
   fires of category F (1000 acres) or greater. This dataset was graciously provided to us by the Climatematch    organizers, saved here: "~/shared/Data/Projects/Wildfires/ClimateAction_countries.shp".


2) MODIS Surface Reflectance 8-Day product (Vermote, 2021) data (more information here https://lpdaac.usgs.gov/products/mod09a1v061/), available here: https://search.earthdata.nasa.gov/search/granules?p=C2343111356-LPCLOUD!C2343111356-LPCLOUD&pg[1][v]=t&pg[1][dnf]=DAY&pg[1][id]=MOD09A1*h19v09*&pg[1][ecd]=2020-01-01T00%3A00%3A00.000Z%2C2020-12-31T23%3A59%3A59.999Z&pg[1][gsk]=-start_date&pg[1][m]=download&pg[1][cd]=f&fi=MODIS&tl=1704799757!3!! (need to register to have access to data)
 
  - we selected the entire 2020 year and tiles (19,09), (20,09), (19,10) and (20,10), which cover Angola, by searching for filenames "MOD09A1*h19v09*", "MOD09A1*h19v10*", "MOD09A1*h20v09*" "and MOD09A1*h20v10*" and all dates in the 2020 year  
  - this resulted in 187 .hdf files (46 or 47 per tile)
  - these images are saved under ~/shared-public/Jintasaurus_Skip_Energico/modis_images 


In [ ]:
# imports
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import matplotlib
from osgeo import gdal
import rioxarray as rxr
from shapely.geometry import Polygon, MultiPolygon
from datetime import datetime, timedelta

# 1. Prepare fires (select dates and polygons)

In [ ]:
# code to retrieve and load the data
url_climateaction = "~/shared/Data/Projects/Wildfires/ClimateAction_countries.shp"
Dataset = gpd.read_file(url_climateaction)  # need to update to OSF and pooch.retrieve

In [ ]:
#angola fires - kamil 14.12.2023
Dataset["IDate"] = pd.to_datetime(Dataset["IDate"])
Dataset["FDate"] = pd.to_datetime(Dataset["FDate"])
# Choose country
country = "Angola"
angolaextent =  (9, 26, -21, -4) #lon,lat
angolalongitude=slice(9,26)
angolalatitudeR=slice(-4,-21) #reversed order for temperature data
angolalatitude=slice(-21,-4) 

# Define the start month (sm) and end month (em) of the dry period (May-September)
year = 2020
angolaYear = Dataset[(Dataset["name"] == country) &
                        (Dataset["IDate"].dt.year == year) & (Dataset["FDate"].dt.year == year)     ]
angola_sorted = angolaYear.sort_values("Area_Acres", ascending = False)
#angola_sorted #angola_sorted
#angolaYear

In [ ]:
vegetationTypes=['Forest','Shrubs','Herbaceous Vegetation']
num_fires = 50
# Sample 50 per veg type fires from 2020 and set a seed for reproducibility
selectionType = ['Random','Largest']
fireselection=selectionType[1] #random or largest?

# angolaYear_sample_combined = gpd.GeoDataFrame(columns=angolaYear.columns) #type(angolaYear) - geopandas.geodataframe.GeoDataFrame

for vegetation in vegetationTypes:
    #the random selection of wildfires
    if fireselection=='Random':
        angolaYear_vegetation = angolaYear[angolaYear["LC_descrip"] == vegetation]
        angolaYear_sample = angolaYear_vegetation.sample(min(500,angolaYear_vegetation.shape[0]), random_state = 42)

    #lets try the largest wildfires
    if fireselection=='Largest':
        angolaYear_vegetation = angola_sorted[angola_sorted["LC_descrip"] == vegetation] #100 largest fires with this vegetation type
        angolaYear_sample = angolaYear_vegetation[0:num_fires]
        if vegetation == 'Forest':
            sample_forest = angolaYear_sample
        elif vegetation == 'Shrubs':
            sample_shrubs = angolaYear_sample
        elif vegetation == 'Herbaceous Vegetation':
            sample_herb = angolaYear_sample
            
    # angolaYear_sample_combined = pd.concat([angolaYear_sample_combined, angolaYear_sample])


In [ ]:
# Function to update min and max from coordinates
def update_min_max(coords, lon_min, lon_max, lat_min, lat_max):
    #global lon_min, lat_min, lon_max, lat_max
    for x, y in coords:
        lon_min = min(lon_min, round(x,2))
        lat_min = min(lat_min, round(y,2))
        lon_max = max(lon_max, round(x,2))
        lat_max = max(lat_max, round(y,2))      
    return lon_min, lon_max, lat_min, lat_max



In [ ]:
#find the fire's size (rectangle with max and min longitude and latitude) from the its geometry polygon
def find_fire_rect(FireData):
    # Initialize max and min values
    lon_min, lat_min= float('inf'), float('inf')
    lon_max, lat_max= float('-inf'), float('-inf')
    # List to store all coordinates
    all_coords = []
        
    for multipolygon in FireData["geometry"]:
        if isinstance(multipolygon, MultiPolygon):
            for polygon in multipolygon.geoms: #the geometry can contain Multypolygon
                # Update min and max from exterior
                lon_min, lon_max, lat_min, lat_max = update_min_max(polygon.exterior.coords, lon_min, lon_max, lat_min, lat_max)

                # Update min and max from interiors
                for interior in polygon.interiors: #the geometry can contain just Polygon
                    lon_min, lon_max, lat_min, lat_max = update_min_max(interior.coords, lon_min, lon_max, lat_min, lat_max)
        elif isinstance(multipolygon, Polygon):
            lon_min, lon_max, lat_min, lat_max = update_min_max(multipolygon.exterior.coords, lon_min, lon_max, lat_min, lat_max)
                
    return {'lon_min': lon_min, 'lon_max': lon_max, 'lat_min':lat_min, 'lat_max':lat_max} # returns Dictionary

In [ ]:
def find_fire_dates(FireData):
    idate = FireData["IDate"]
    fdate = FireData["FDate"]

    idatestr = idate.dt.strftime('%Y-%m-%d').iloc[0] #day when the fire started 
    fdatestr = fdate.dt.strftime('%Y-%m-%d').iloc[0] #day when the fire finished 

    # our precipitation data are samplet just once in a month - the first day of it
    idateMstr = idate.dt.strftime('%Y-%m-01').iloc[0] #month (first day of it) when the fire started 
    fdateMstr = fdate.dt.strftime('%Y-%m-01').iloc[0] #month (first day of it) when the fire finished 
    return {'idatestr':idatestr,'idateMstr':idateMstr, 'fdatestr':fdatestr, 'fdateMstr':fdateMstr}  # returns Dictionary


# 2. Load images for selected fires (pre and post)

In [ ]:
def extract_date_from_filename(filename):
    # Split the filename and extract the part with the date
    date_str = filename.split('.')[1]  # 'A2020353' in your example

    # Extract the year and the day of the year
    year = int(date_str[1:5])  # '2020' in your example
    day_of_year = int(date_str[5:])  # '353' in your example

    # Convert to a date
    date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)

    return date.strftime('%Y-%m-%d')

# Example usage
filename = 'MOD09A1.A2020001.h19v09.061.2020324110240.hdf'
print(extract_date_from_filename(filename))  # Output: '2020-01-01'


In [ ]:
def nearest_after(items, pivot):
    #calculate the diff. for all dates with the "pivot" (date of image)
    difference_dates = items - pivot
    
    #convert to int
    diff_int = np.array([d.days for d in difference_dates])
    
    #take only the positive differences, i.e., dates after date of image - rest is 999
    diff_int[np.argwhere(diff_int<=0)] = 999
    
    #take the min, i.e, closest date 
    closest_date = items[np.argmin(diff_int)]
        
    return closest_date.strftime('%Y-%m-%d')

In [ ]:
def nearest_before(items, pivot):
    #calculate the diff. for all dates with the "pivot" (date of image)
    difference_dates = items - pivot
    
    #convert to int
    diff_int = np.array([d.days for d in difference_dates])
    
    #take only the negative differences, i.e., dates before date of image - rest is 999
    diff_int[np.argwhere(diff_int>=0)] = 999
    
    #take the min, i.e, closest date 
    closest_date = items[np.argmin(abs(diff_int))]
    
    
    return closest_date.strftime('%Y-%m-%d')

In [ ]:
def computeNDVI(image_R, image_NIR, mask_R, mask_NIR):
    r,c = image_R.shape
    ndviOutput = np.zeros((r,c))
    for x in range(c):
        for y in range(r):
            if (image_NIR[y,x] == 0 and image_R[y,x] == 0) or (mask_NIR[y,x] == 1 and mask_R[y,x] == 1):
                ndviOutput[y,x] = np.nan
            else:
                ndviOutput[y,x] = (image_NIR[y,x] - image_R[y,x]) / (image_NIR[y,x] + image_R[y,x])

    return ndviOutput


In [ ]:
def preprocess_data(image):


    # Bands 1 (Red), 4 (Green), 3 (Blue), and 2 (Near Infrared) for MOD09A1 RGB
    # modis_pre["sur_refl_b01"][0,:,:]  # red
    # modis_pre["sur_refl_b04"][0,:,:]  # green
    # modis_pre["sur_refl_b03"][0,:,:]  # blue
    # modis_pre["sur_refl_b02"][0,:,:]  # Near Infrared (NIR)
    # modis_pre["sur_refl_b07"][0,:,:]  # SWIR band

    scale_factor = 0.0001
    fill_value   = -28672

    #---------------------------------------------------------------------------------------
    # Read data from the specified band and convert it to a NumPy array of float64 data type
    #---------------------------------------------------------------------------------------
    data_R    = image["sur_refl_b01"][0,:,:] #.ReadAsArray().astype(np.float64)
    data_G    = image["sur_refl_b04"][0,:,:]
    data_B    = image["sur_refl_b03"][0,:,:]
    data_NIR  = image["sur_refl_b02"][0,:,:]
    data_SWIR = image["sur_refl_b07"][0,:,:]


    # Close the GDAL dataset objects to free up memory
    #values, values2 = None, None

    #--------------------------------------------------------
    # Replace fill values with NaN (Not a Number) in the data
    #--------------------------------------------------------
    data_R    = np.where(data_R == fill_value,    np.nan, data_R)
    data_G    = np.where(data_G == fill_value,    np.nan, data_G)
    data_B    = np.where(data_B == fill_value,    np.nan, data_B)
    data_NIR  = np.where(data_NIR == fill_value,  np.nan, data_NIR)
    data_SWIR = np.where(data_SWIR == fill_value, np.nan, data_SWIR)


    #-------------------------
    # Multiply by scale factor
    #-------------------------
    data_R    = data_R    * scale_factor
    data_G    = data_G    * scale_factor
    data_B    = data_B    * scale_factor
    data_NIR  = data_NIR  * scale_factor
    data_SWIR = data_SWIR * scale_factor

    return data_R, data_NIR, data_SWIR

In [ ]:
def showSCL(image): #view SCL imagery
    cmap = matplotlib.colors.ListedColormap(
        [
            "black",
            "red",
            "chocolate",
            "brown",
            "lime",
            "yellow",
            "blue",
            "aqua",
            "darkgrey",
            "lightgrey",
            "deepskyblue",
            "magenta",
        ]
    )
    plt.imshow(image, cmap=cmap)
    plt.colorbar()

    plt.show()


In [ ]:
def computeSCLMask(image): # compute SCL Mask to 0s and 1s, masking out clouds and bad pixels
    rImage, cImage = image.shape
    sclOutput = np.zeros((rImage, cImage))
    for x in range(cImage):
        for y in range(rImage):
            sclOutput[y, x] = 1 if image[y, x] in [0, 1, 3, 8, 9, 11] else 0

    return sclOutput

In [ ]:
# use this code to remap the NDVI to specific colors for values
def remapNDVI(NDVI):
    remapped = np.zeros((NDVI.shape[0], NDVI.shape[1]))
    for x in range(remapped.shape[0]):
        for y in range(remapped.shape[1]):
            if np.isnan(NDVI[x, y]):
                remapped[x, y] = np.nan
            elif NDVI[x, y] <= -0.2:
                remapped[x, y] = 1
            elif NDVI[x, y] <= 0:
                remapped[x, y] = 2
            elif NDVI[x, y] <= 0.1:
                remapped[x, y] = 3
            elif NDVI[x, y] <= 0.2:
                remapped[x, y] = 4
            elif NDVI[x, y] <= 0.3:
                remapped[x, y] = 5
            elif NDVI[x, y] <= 0.4:
                remapped[x, y] = 6
            elif NDVI[x, y] <= 0.5:
                remapped[x, y] = 7
            elif NDVI[x, y] <= 0.6:
                remapped[x, y] = 8
            elif NDVI[x, y] <= 0.7:
                remapped[x, y] = 9
            elif NDVI[x, y] <= 0.8:
                remapped[x, y] = 10
            elif NDVI[x, y] <= 0.9:
                remapped[x, y] = 11
            elif NDVI[x, y] <= 1:
                remapped[x, y] = 12
            else:
                remapped[x, y] = 13
    return remapped

In [ ]:
def showNDVI(image): # view remapped NDVI
    cmap = matplotlib.colors.ListedColormap(
        [
            "#000000",
            "#a50026",
            "#d73027",
            "#f46d43",
            "#fdae61",
            "#fee08b",
            "#ffffbf",
            "#d9ef8b",
            "#a6d96a",
            "#66bd63",
            "#1a9850",
            "#006837",
        ]
    )
    plt.imshow(image, cmap=cmap)
    plt.colorbar()

    plt.show()

In [ ]:
def save_NDVI(image,filename,modis_lon_min_pre,modis_lon_max_pre,modis_lat_min_pre,modis_lat_max_pre): # view remapped NDVI
    cmap = matplotlib.colors.ListedColormap(
        [
            "#000000",
            "#a50026",
            "#d73027",
            "#f46d43",
            "#fdae61",
            "#fee08b",
            "#ffffbf",
            "#d9ef8b",
            "#a6d96a",
            "#66bd63",
            "#1a9850",
            "#006837",
        ]
    )

    
    ax=plt.imshow(image, cmap=cmap, extent=[round(modis_lon_min_pre),round(modis_lon_max_pre),round(modis_lat_min_pre*-1),round(modis_lat_max_pre*-1)], origin='lower')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°S)')
    cbar = plt.colorbar()
    cbar.set_label('remapped NDVI (a.u)')
    plt.savefig(filename)


In [ ]:
# Dataset-specific functions: NBR and dNBR Code
# This code creates the NBR for each image then uses the NBR to create the dNBR. It can easily be updated for other burnt area indices
''' Like the previous indices, this code leverages Bands 8 and Bands 12 to calculate the NBR,
then it takes the PreNBR and subtracts the PostNBR. By doing so, it removes pixels that had values that were unchanged.
After those pixels are removed, the only pixels left are those with values that have changed - ie: the pixels that experienced the fire.
How drastic the change between the values of the Pre and Post NBR pixels indicate the severity of the fire'''
def computeFireMasks(pre_fire_NIR, pre_fire_SWIR, post_fire_NIR, post_fire_SWIR):

    rows, columns = pre_fire_NIR.shape
    nbrPost = np.zeros((rows, columns))
    nbrPre = np.zeros((rows, columns))
    dnbr = np.zeros((rows,columns))

    for x in range(columns):
        for y in range(rows):
            nbrPost[y, x] = (post_fire_NIR[y, x] - post_fire_SWIR[y, x])/(post_fire_NIR[y, x] + post_fire_SWIR[y, x])
            nbrPre[y, x] = (pre_fire_NIR[y, x] - pre_fire_SWIR[y, x])/(pre_fire_NIR[y, x] + pre_fire_SWIR[y, x])
            dnbr[y, x] = nbrPre[y, x] - nbrPost[y, x]

    return dnbr



In [ ]:
def remapDNBR(dnbr):
    # this code applies a threshold to the dNBR to show the level of burn intensity (unburned, low severity, moderate severity, or high severity)
    remapped = np.zeros((dnbr.shape[0], dnbr.shape[1]))
    for x in range(remapped.shape[0]):
        for y in range(remapped.shape[1]):
            if np.isnan(dnbr[x, y]):
                remapped[x, y] = np.nan
            elif dnbr[x, y] <= -0.251:
                remapped[x, y] = 1
            elif dnbr[x, y] <= -0.101:
                remapped[x, y] = 2
            elif dnbr[x, y] <= 0.099:
                remapped[x, y] = 3
            elif dnbr[x, y] <= 0.269:
                remapped[x, y] = 4
            elif dnbr[x, y] <= 0.439:
                remapped[x, y] = 5
            elif dnbr[x, y] <= 0.659:
                remapped[x, y] = 6
            elif dnbr[x, y] <= 1.3:
                remapped[x, y] = 7
            else:
                remapped[x, y] = 8
    return remapped

In [ ]:
def showDNBR(dnbr):
    cmap = matplotlib.colors.ListedColormap(
        [
            "blue",
            "teal",
            "green",
            "yellow",
            "orange",
            "red",
            "purple",
        ]
    )
    plt.imshow(remapDNBR(dnbr), cmap=cmap)
    plt.colorbar()

    plt.show()

In [ ]:
def save_DNBR(dnbr,filename,modis_lon_min_pre,modis_lon_max_pre,modis_lat_min_pre,modis_lat_max_pre):
    cmap = matplotlib.colors.ListedColormap(
        [
            "blue",
            "teal",
            "green",
            "yellow",
            "orange",
            "red",
            "purple",
        ]
    )

    ax=plt.imshow(remapDNBR(dnbr), cmap=cmap, extent=[round(modis_lon_min_pre),round(modis_lon_max_pre),round(modis_lat_min_pre*-1),round(modis_lat_max_pre*-1)], origin='lower')
    plt.xlabel('Longitude (°E)')
    plt.ylabel('Latitude (°S)')
    cbar = plt.colorbar()
    cbar.set_label('remapped dNBR (a.u)')
    
    plt.savefig(filename)

In [ ]:
def get_modis_latitude_longitude(modis_dataset):

    num_rows = len(modis_dataset.y)
    num_columns = len(modis_dataset.x)

    modis_lat = modis_dataset.attrs["GRINGPOINTLATITUDE.1"].split(',')
    modis_lon = modis_dataset.attrs["GRINGPOINTLONGITUDE.1"].split(',')

    modis_lat_west_jump = (float(modis_lat[1]) - float(modis_lat[0]))/num_rows #rows
    modis_lat_east_jump = (float(modis_lat[2]) - float(modis_lat[3]))/num_rows
    modis_lat_jump = np.mean([modis_lat_west_jump, modis_lat_east_jump])

    modis_lon_north_jump = (float(modis_lon[2]) - float(modis_lon[1]))/num_columns #columns
    modis_lon_south_jump = (float(modis_lon[3]) - float(modis_lon[0]))/num_columns #columns
    modis_lon_jump = np.mean([modis_lon_north_jump, modis_lon_south_jump])

    modis_min_lon = np.mean([float(modis_lon[0]),float(modis_lon[1])])
    modis_max_lon = np.mean([float(modis_lon[2]),float(modis_lon[3])])
    modis_min_lat = np.mean([float(modis_lat[0]),float(modis_lat[3])])
    modis_max_lat = np.mean([float(modis_lat[1]),float(modis_lat[2])])
    
    return modis_min_lon, modis_max_lon, modis_min_lat, modis_max_lat, modis_lon_jump, modis_lat_jump
                  

In [ ]:
def fire_rect_indices(modis_lon_min,modis_lon_jump,modis_lat_max,modis_lat_jump,rect):
    
    index_lon_min = round((rect["lon_min"] - modis_lon_min)/modis_lon_jump)
    index_lon_max = round((rect["lon_max"] - modis_lon_min)/modis_lon_jump)
    index_lat_min = abs(round((rect["lat_min"] - modis_lat_max)/modis_lat_jump)) #abs because lat is negative
    index_lat_max = abs(round((rect["lat_max"] - modis_lat_max)/modis_lat_jump)) 
    
    return index_lon_min, index_lon_max, index_lat_min, index_lat_max

In [ ]:
#get the Modis dates
dates_modis_19_09 = []
dates_modis_datetime_19_09 = []
all_filenames_19_09 = []

dates_modis_19_10 = []
dates_modis_datetime_19_10 = []
all_filenames_19_10 = []

dates_modis_20_09 = []
dates_modis_datetime_20_09 = []
all_filenames_20_09 = []

dates_modis_20_10 = []
dates_modis_datetime_20_10 = []
all_filenames_20_10 = []

modis_dir=os.path.expanduser("~/shared-public/Jintasaurus_Skip_Energico/modis_images/")

for filename in os.listdir(modis_dir):
    if ".hdf" in filename and "h19v09" in filename:
        date_filename = extract_date_from_filename(filename)
        d_datetime = datetime.strptime(date_filename, "%Y-%m-%d").date()
        dates_modis_19_09.append(date_filename)
        dates_modis_datetime_19_09.append(d_datetime)
        all_filenames_19_09.append(os.path.join(modis_dir, filename))
    
    elif ".hdf" in filename and "h19v10" in filename:
        date_filename = extract_date_from_filename(filename)
        d_datetime = datetime.strptime(date_filename, "%Y-%m-%d").date()
        dates_modis_19_10.append(date_filename)
        dates_modis_datetime_19_10.append(d_datetime)
        all_filenames_19_10.append(os.path.join(modis_dir, filename))
        
    elif ".hdf" in filename and "h20v09" in filename:
        date_filename = extract_date_from_filename(filename)
        d_datetime = datetime.strptime(date_filename, "%Y-%m-%d").date()
        dates_modis_20_09.append(date_filename)
        dates_modis_datetime_20_09.append(d_datetime)
        all_filenames_20_09.append(os.path.join(modis_dir, filename))
    
    elif ".hdf" in filename and "h20v10" in filename:
        date_filename = extract_date_from_filename(filename)
        d_datetime = datetime.strptime(date_filename, "%Y-%m-%d").date()
        dates_modis_20_10.append(date_filename)
        dates_modis_datetime_20_10.append(d_datetime)
        all_filenames_20_10.append(os.path.join(modis_dir, filename))


In [ ]:
save_fire_index = 1

for vegetation in vegetationTypes:
    
    ndvi_pre_remapped = []
    ndvi_post_remapped = []
    dndvi_remapped = []
    dnbr_remapped = [] 
    ndvi_pre_og = []
    dnbr_og = []

    for j in range(num_fires):
        if vegetation == 'Forest':
            OneFire = sample_forest[j:j+1]
        elif vegetation == 'Shrubs':
            OneFire = sample_shrubs[j:j+1]
        if vegetation == 'Herbaceous Vegetation':
            OneFire = sample_herb[j:j+1]
            
        print(vegetation)
        firedate = find_fire_dates(OneFire)
        rect = find_fire_rect(OneFire)  
        
        #select tile
        if rect['lat_min'] > -10 and rect['lon_max'] < 20:
            dates_modis = np.array(dates_modis_19_09)
            dates_modis_dt = np.array(dates_modis_datetime_19_09)
            all_filenames = all_filenames_19_09
        elif rect['lat_min'] > -10 and rect['lon_max'] > 20:
            dates_modis = np.array(dates_modis_20_09)
            dates_modis_dt = np.array(dates_modis_datetime_20_09)
            all_filenames = all_filenames_20_09
        elif rect['lat_min'] < -10 and rect['lon_max'] < 20:
            dates_modis = np.array(dates_modis_19_10)
            dates_modis_dt = np.array(dates_modis_datetime_19_10)
            all_filenames = all_filenames_19_10
        elif rect['lat_min'] < -10 and rect['lon_max'] > 20:
            dates_modis = np.array(dates_modis_20_10)
            dates_modis_dt = np.array(dates_modis_datetime_20_10)
            all_filenames = all_filenames_20_10

        #beginning date
        date_start = datetime.strptime(firedate['idatestr'],"%Y-%m-%d").date()
        date_modis_start = nearest_before(dates_modis_dt, date_start)
        print('Start date: ' + str(date_start))
        print('Modis start date: '+ str(date_modis_start))

        #end date
        date_end = datetime.strptime(firedate['fdatestr'],"%Y-%m-%d").date()
        date_modis_end = nearest_after(dates_modis_dt, date_end)
        print('End date: ' + str(date_end))
        print('Modis end date: ' + str(date_modis_end))

        #load images
        print('Loading images')
        start_filename = all_filenames[np.argwhere(dates_modis==date_modis_start)[0][0]]
        end_filename = all_filenames[np.argwhere(dates_modis==date_modis_end)[0][0]]

        modis_pre  = rxr.open_rasterio(start_filename, masked = True)
        modis_post = rxr.open_rasterio(end_filename, masked = True)     

        modis_lon_min_pre, modis_lon_max_pre, modis_lat_min_pre, modis_lat_max_pre, modis_lon_jump_pre, modis_lat_jump_pre = get_modis_latitude_longitude(modis_pre)
        modis_lon_min_post, modis_lon_max_post, modis_lat_min_post, modis_lat_max_post, modis_lon_jump_post, modis_lat_jump_post = get_modis_latitude_longitude(modis_post)

        data_pre_R, data_pre_NIR, data_pre_SWIR = preprocess_data(modis_pre)
        data_post_R, data_post_NIR, data_post_SWIR = preprocess_data(modis_post)

        print('show images')
        showSCL(data_pre_R)
        showSCL(data_post_R)
        showSCL(data_pre_NIR)
        showSCL(data_post_NIR)

        #cloud masks
        print('making masks')
        pre_SCL_Mask_R   = computeSCLMask(data_pre_R)
        pre_SCL_Mask_NIR = computeSCLMask(data_pre_NIR)

        post_SCL_Mask_R   = computeSCLMask(data_post_R)
        post_SCL_Mask_NIR = computeSCLMask(data_post_NIR)

        pre_SCL_Mask_SWIR  = computeSCLMask(data_pre_SWIR)
        post_SCL_Mask_SWIR = computeSCLMask(data_post_SWIR)

        #NDVI
        print('calculating NDVI')
        ndvi_pre  = computeNDVI(data_pre_R,  data_pre_NIR,  pre_SCL_Mask_R,  pre_SCL_Mask_NIR)
        ndvi_post = computeNDVI(data_post_R, data_post_NIR, post_SCL_Mask_R, post_SCL_Mask_NIR)
        dNDVI = ndvi_pre - ndvi_post

        print('remapping NDVI')
        ndvi_pre_remapped_f = remapNDVI(ndvi_pre) #select rectangle
        ndvi_post_remapped_f = remapNDVI(ndvi_post)
        dndvi_remapped_f = remapNDVI(dNDVI)

        print('show NDVI')
        showNDVI(ndvi_pre_remapped_f)
        showNDVI(ndvi_post_remapped_f)
        showNDVI(dndvi_remapped_f)

        #DNBR
        print('calculate dnbr')
        dnbr = computeFireMasks(data_pre_NIR, data_pre_SWIR, data_post_NIR, data_post_SWIR)
        remapped_dnbr = remapDNBR(dnbr)

        print('show dnbr')
        showDNBR(dnbr) #remap function is within this

        #get fire rectangle
        print('get fire rectangle')
        index_lon_min_pre, index_lon_max_pre, index_lat_min_pre, index_lat_max_pre = fire_rect_indices(modis_lon_min_pre,
                                                                                       modis_lon_jump_pre,
                                                                                       modis_lat_max_pre,
                                                                                       modis_lat_jump_pre,
                                                                                       rect)
        index_lon_min_post, index_lon_max_post, index_lat_min_post, index_lat_max_post = fire_rect_indices(modis_lon_min_post,
                                                                                       modis_lon_jump_post,
                                                                                       modis_lat_max_post,
                                                                                       modis_lat_jump_post,
                                                                                       rect)


        #avg over pre and post
        avg_idx_lon_min = round(np.mean([index_lon_min_pre,index_lon_min_post]))
        avg_idx_lon_max = round(np.mean([index_lon_max_pre,index_lon_max_post]))
        avg_idx_lat_min = round(np.mean([index_lat_min_pre,index_lat_min_post]))
        avg_idx_lat_max = round(np.mean([index_lat_max_pre,index_lat_max_post]))

        #get ndvi and dnbr only for fire rectangle
        rect_ndvi_pre = ndvi_pre_remapped_f[index_lat_max_pre:index_lat_min_pre+1,index_lon_min_pre:index_lon_max_pre+1]
        rect_ndvi_post = ndvi_post_remapped_f[index_lat_max_post:index_lat_min_post+1,index_lon_min_post:index_lon_max_post+1]
        rect_dndvi = dndvi_remapped_f[avg_idx_lat_max:avg_idx_lat_min+1,avg_idx_lon_min:avg_idx_lon_max+1]
        rect_dnbr = remapped_dnbr[avg_idx_lat_max:avg_idx_lat_min+1,avg_idx_lon_min:avg_idx_lon_max+1]
        
        #nonremapped
        rect_ndvi_pre_og = ndvi_pre[index_lat_max_pre:index_lat_min_pre+1,index_lon_min_pre:index_lon_max_pre+1]
        rect_dnbr_og = dnbr[avg_idx_lat_max:avg_idx_lat_min+1,avg_idx_lon_min:avg_idx_lon_max+1]
    
        #append 
        ndvi_pre_remapped.append(np.nanmean(rect_ndvi_pre))  
        ndvi_post_remapped.append(np.nanmean(rect_ndvi_post))     
        dndvi_remapped.append(np.nanmean(rect_dndvi))          
        dnbr_remapped.append(np.nanmean(rect_dnbr))
        
        #nonremapped
        ndvi_pre_og.append(np.nanmean(rect_ndvi_pre_og))
        dnbr_og.append(np.nanmean(rect_dnbr_og))

        #save - keep rewriting at every iteration
        np.save('ndvi_pre_remapped_largest_fires_redone_{}'.format(vegetation),ndvi_pre_remapped)
        np.save('ndvi_post_remapped_largest_fires_redone_{}'.format(vegetation),ndvi_post_remapped)
        np.save('dndvi_remapped_largest_fires_redone_{}'.format(vegetation),dndvi_remapped)
        np.save('dnbr_remapped_largest_fires_redone_{}'.format(vegetation),dnbr_remapped)
        np.save('ndvi_pre_og_largest_fires_redone_{}'.format(vegetation),ndvi_pre_og)
        np.save('dnbr_og_largest_fires_redone_{}'.format(vegetation),dnbr_og)
        
        #if wanted, can save the images (NDVI/DNBR)
        if j == save_fire_index:
            filename_ndvi = 'ndvi_pre_remapped.png'
            save_NDVI(ndvi_pre_remapped,filename_ndvi,modis_lon_min_pre,modis_lon_max_pre,modis_lat_min_pre,modis_lat_max_pre)
            filename_dnbr = 'dnbr_remapped.png'
            save_DNBR(dnbr,filename_dnbr,modis_lon_min_pre,modis_lon_max_pre,modis_lat_min_pre,modis_lat_max_pre)
            
        print('end of loop')
        del modis_pre
        del modis_post


-------------------------------------------------